# Signal Mapping Fundamentals

## Purpose

This notebook teaches you how to map signals from point cloud data to voxel grids using various interpolation methods. You'll learn to use nearest neighbor, linear, IDW, KDE, and RBF interpolation with interactive widgets.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Understand signal mapping concept and importance
- ✅ Use all interpolation methods interactively
- ✅ Compare method performance and accuracy in real-time
- ✅ Select appropriate method for your use case
- ✅ Understand parameter effects on mapping quality

## Estimated Duration

60-90 minutes

---

## Overview

Signal mapping is the process of interpolating point cloud data (with signals) onto a regular voxel grid. The AM-QADF framework supports multiple interpolation methods:

- 🎯 **Nearest Neighbor**: Fast, assigns each voxel to its nearest point
- 📈 **Linear**: Smooth interpolation using k-nearest neighbors
- ⚖️ **IDW (Inverse Distance Weighting)**: Distance-weighted interpolation
- 📊 **Gaussian KDE**: Kernel density estimation for smooth surfaces
- 🌟 **RBF (Radial Basis Functions)**: Exact interpolation at data points with smooth interpolation between

Use the interactive widgets below to explore signal mapping - no coding required!


In [1]:
# Setup: Import required libraries
import sys
import time
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
from typing import Optional, Tuple, Dict, Any

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import infrastructure
INFRASTRUCTURE_AVAILABLE = False
try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

# Try to import signal mapping classes
SIGNAL_MAPPING_AVAILABLE = False
try:
    from am_qadf.signal_mapping.execution.sequential import interpolate_to_voxels
    from am_qadf.signal_mapping.methods.nearest_neighbor import NearestNeighborInterpolation
    from am_qadf.signal_mapping.methods.linear import LinearInterpolation
    from am_qadf.signal_mapping.methods.idw import IDWInterpolation
    from am_qadf.signal_mapping.methods.kde import GaussianKDEInterpolation
    from am_qadf.signal_mapping.methods.rbf import RBFInterpolation
    SIGNAL_MAPPING_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Signal mapping classes not available: {e} - using demo mode")

# Try to import VoxelGrid
VOXEL_AVAILABLE = False
try:
    from am_qadf.voxelization.voxel_grid import VoxelGrid
    VOXEL_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ VoxelGrid not available: {e} - using demo mode")

# Try to import query clients for loading data
QUERY_CLIENTS_AVAILABLE = False
mongo_client = None
unified_client = None
voxel_storage = None
alignment_storage = None

# Try to import alignment storage
ALIGNMENT_STORAGE_AVAILABLE = False
try:
    from am_qadf.synchronization import AlignmentStorage
    ALIGNMENT_STORAGE_AVAILABLE = True
except ImportError as e:
    ALIGNMENT_STORAGE_AVAILABLE = False
    print(f"⚠️ AlignmentStorage not available: {e}")

if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        if mongo_client and mongo_client.is_connected():
            from am_qadf.query import UnifiedQueryClient
            unified_client = UnifiedQueryClient(mongo_client=mongo_client)
            QUERY_CLIENTS_AVAILABLE = True
            print("✅ MongoDB connection established")
            
            # Initialize alignment storage if available
            if ALIGNMENT_STORAGE_AVAILABLE:
                try:
                    alignment_storage = AlignmentStorage(mongo_client=mongo_client)
                    print("✅ AlignmentStorage initialized")
                except Exception as e:
                    print(f"⚠️ AlignmentStorage initialization failed: {e}")
            
            # Try to import voxel grid storage
            try:
                import importlib.util
                voxel_storage_path = src_dir / 'am_qadf' / 'voxel_domain' / 'voxel_storage.py'
                if voxel_storage_path.exists():
                    spec = importlib.util.spec_from_file_location("voxel_storage", voxel_storage_path)
                    voxel_storage_module = importlib.util.module_from_spec(spec)
                    spec.loader.exec_module(voxel_storage_module)
                    VoxelGridStorage = voxel_storage_module.VoxelGridStorage
                    voxel_storage = VoxelGridStorage(mongo_client)
                    print("✅ VoxelGridStorage initialized")
            except Exception as e:
                print(f"⚠️ VoxelGridStorage not available: {e}")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")

# Check for Spark and Parallel processing availability
SPARK_AVAILABLE = False
PARALLEL_AVAILABLE = False

# Check PySpark availability
try:
    from pyspark.sql import SparkSession
    from am_qadf.signal_mapping.utils.spark_utils import create_spark_session
    SPARK_AVAILABLE = True
    print("✅ PySpark available - Spark-based signal mapping enabled")
except ImportError as e:
    print(f"⚠️ PySpark not available: {e} - Spark-based mapping disabled")

# Check parallel processing availability
try:
    from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
    import multiprocessing
    PARALLEL_AVAILABLE = True
    cpu_count = multiprocessing.cpu_count()
    print(f"✅ Parallel processing available - {cpu_count} CPU cores detected")
except Exception as e:
    print(f"⚠️ Parallel processing not available: {e}")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ MongoDB connection established
✅ AlignmentStorage initialized
✅ VoxelGridStorage initialized
✅ PySpark available - Spark-based signal mapping enabled
✅ Parallel processing available - 8 CPU cores detected
✅ Setup complete!


## Interactive Signal Mapping Interface

Use the widgets below to map signals to voxel grids. Select interpolation method, configure parameters, and visualize results interactively!


In [2]:
# Create Interactive Signal Mapping Interface

# Global state
current_points = None
current_signals = None
current_grid = None
mapped_grid = None
sample_data_generated = False
current_model_id = None
current_grid_id = None
loaded_grid_data = None

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_sample_data(n_points=1000, bbox_min=(-50, -50, 0), bbox_max=(50, 50, 100)):
    """Generate sample point cloud data with signals."""
    np.random.seed(42)
    
    # Generate random points in bounding box
    points = np.random.uniform(
        low=bbox_min,
        high=bbox_max,
        size=(n_points, 3)
    )
    
    # Generate signals
    # Signal 1: Temperature (smooth gradient)
    x_norm = (points[:, 0] - bbox_min[0]) / (bbox_max[0] - bbox_min[0])
    y_norm = (points[:, 1] - bbox_min[1]) / (bbox_max[1] - bbox_min[1])
    z_norm = (points[:, 2] - bbox_min[2]) / (bbox_max[2] - bbox_min[2])
    
    temperature = 500 + 300 * np.sin(2 * np.pi * x_norm) * np.cos(2 * np.pi * y_norm) + 200 * z_norm
    temperature += np.random.normal(0, 10, n_points)  # Add noise
    
    # Signal 2: Power (with hotspots)
    power = 200 + 100 * np.exp(-((points[:, 0] - 10)**2 + (points[:, 1] - 10)**2) / 100)
    power += 50 * np.exp(-((points[:, 0] + 20)**2 + (points[:, 1] + 20)**2) / 200)
    power += np.random.normal(0, 5, n_points)
    
    # Signal 3: Density (layered pattern)
    density = 0.8 + 0.2 * np.sin(2 * np.pi * z_norm * 5) + np.random.normal(0, 0.02, n_points)
    
    signals = {
        'temperature': temperature,
        'power': power,
        'density': density
    }
    
    return points, signals

# ============================================
# Top Panel: Method Selection and Actions
# ============================================

method_label = widgets.HTML("<b>Interpolation Method:</b>")
method_selector = RadioButtons(
    options=[
        ('Nearest Neighbor', 'nearest'),
        ('Linear', 'linear'),
        ('IDW (Inverse Distance Weighting)', 'idw'),
        ('Gaussian KDE', 'gaussian_kde'),
        ('RBF (Radial Basis Functions)', 'rbf')
    ],
    value='nearest',
    description='Method:',
    style={'description_width': 'initial'}
)

# Model and Grid Selection (for MongoDB)
model_label = widgets.HTML("<b>Model & Grid:</b>")
models = []
model_options = [("━━━ Select Model ━━━", None)]

if unified_client and mongo_client:
    try:
        from am_qadf.query import STLModelClient
        stl_client = STLModelClient(mongo_client=mongo_client)
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
        if len(model_options) == 1:
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))

model_dropdown = Dropdown(
    options=model_options,
    value=None,  # Default to "Select Model" option
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px', display='flex')
)

# Grid selection (populated when model is selected)
grid_options = [("━━━ Select Grid ━━━", None)]

grid_dropdown = Dropdown(
    options=grid_options,
    value=None,
    description='Empty Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px')
)

# Dropdown for selecting mapped grid (initially hidden)
mapped_grid_dropdown = Dropdown(
    options=[("━━━ Select Mapped Grid ━━━", None)],
    value=None,
    description='Mapped Grid:',
    style={'description_width': 'initial'},
    layout=Layout(width='300px', display='none')
)

# Signal data source selection
signal_source_label = widgets.HTML("<b>Signal Data:</b>")
signal_source_selector = RadioButtons(
    options=[
        ('Laser Parameters', 'laser'),
        ('ISPM Monitoring', 'ispm'),
        ('Hatching Paths', 'hatching'),
        ('CT Scan', 'ct'),
        ('All Sources', 'all')
    ],
    value='all',  # Default to 'all' to query all sources by default
    description='Source:',
    style={'description_width': 'initial'}
)

map_button = Button(
    description='Map Signals',
    button_style='success',
    icon='map',
    layout=Layout(width='150px')
)

save_mapped_button = Button(
    description='Save Mapped Grid',
    button_style='info',
    icon='save',
    layout=Layout(width='150px')
)

load_mapped_button = Button(
    description='Load Mapped Grid',
    button_style='',
    icon='folder-open',
    layout=Layout(width='150px')
)

compare_button = Button(
    description='Compare Methods',
    button_style='',
    icon='copy',
    layout=Layout(width='150px')
)


# Create a visually organized top panel with clear sections and column layouts
top_panel = VBox([
    # Section 1: Interpolation Method Selection (3 columns)
    widgets.HTML("<div style='background: #f0f0f0; padding: 8px; border-radius: 4px; margin-bottom: 5px;'><b>📊 Interpolation Method</b></div>"),
    HBox([
        VBox([
            widgets.HTML("<div style='padding: 3px; font-size: 11px;'><b>Fast Methods:</b></div>"),
            method_selector
        ], layout=Layout(width='33%', padding='5px')),
        VBox([
            widgets.HTML("<div style='padding: 3px; font-size: 11px;'><b>Distance-Based:</b></div>"),
            widgets.HTML("<div style='padding: 3px; color: #666; font-size: 10px;'>Nearest, Linear, IDW</div>")
        ], layout=Layout(width='33%', padding='5px')),
        VBox([
            widgets.HTML("<div style='padding: 3px; font-size: 11px;'><b>Advanced:</b></div>"),
            widgets.HTML("<div style='padding: 3px; color: #666; font-size: 10px;'>KDE, RBF</div>")
        ], layout=Layout(width='33%', padding='5px'))
    ], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px', width='100%')),
    
    # Section 2: Data Selection (2 columns)
    widgets.HTML("<div style='background: #f0f0f0; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>🗂️ Data Selection</b></div>"),
    HBox([
        # Column 1: Model and Grid
        VBox([
            widgets.HTML("<div style='padding: 5px;'><b>Model & Empty Grid:</b></div>"),
            VBox([
                HBox([
                    widgets.HTML("<div style='padding: 2px; font-size: 11px; color: #555;'>Model:</div>"),
                    model_dropdown
                ], layout=Layout(justify_content='flex-start', gap='5px', width='100%')),
                HBox([
                    widgets.HTML("<div style='padding: 2px; font-size: 11px; color: #555;'>Empty Grid:</div>"),
                    grid_dropdown
                ], layout=Layout(justify_content='flex-start', gap='5px', width='100%'))
            ], layout=Layout(width='100%', gap='8px'))
        ], layout=Layout(width='50%', padding='8px')),
        
        # Column 2: Signal Source
        VBox([
            widgets.HTML("<div style='padding: 5px;'><b>Signal Source:</b></div>"),
            signal_source_selector
        ], layout=Layout(width='50%', padding='8px'))
    ], layout=Layout(justify_content='flex-start', padding='5px', margin='5px 0px', width='100%')),
    
    # Section 3: Load Existing Mapped Grid
    widgets.HTML("<div style='background: #f0f0f0; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>📂 Load Existing Mapped Grid</b></div>"),
    HBox([
        mapped_grid_dropdown
    ], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px', width='100%')),
    
    # Section 4: Actions
    widgets.HTML("<div style='background: #e8f4f8; padding: 8px; border-radius: 4px; margin: 10px 0px 5px 0px;'><b>⚡ Actions</b></div>"),
    HBox([
        map_button,
        save_mapped_button,
        load_mapped_button,
        compare_button
    ], layout=Layout(justify_content='flex-start', padding='8px', margin='5px 0px', gap='10px', width='100%'))
], layout=Layout(
    padding='15px',
    border='2px solid #ddd',
    border_radius='8px',
    background='#fafafa',
    margin='10px 0px',
    width='100%'
))

# ============================================
# Left Panel: Method Parameters
# ============================================

# Method-specific parameter sections
params_label = widgets.HTML("<b>Method Parameters:</b>")

# Nearest Neighbor parameters (none needed, but show info)
nn_info = widgets.HTML("<p>No parameters required. Fastest method.</p>")
nn_params = VBox([nn_info], layout=Layout(display='flex'))

# Linear parameters
linear_k_neighbors = IntSlider(value=8, min=1, max=50, step=1, description='K Neighbors:', style={'description_width': 'initial'})
linear_radius = FloatSlider(value=10.0, min=0.1, max=100.0, step=0.1, description='Radius (mm):', style={'description_width': 'initial'})
linear_use_radius = Checkbox(value=False, description='Use Radius Limit', style={'description_width': 'initial'})
linear_params = VBox([
    linear_k_neighbors,
    linear_use_radius,
    linear_radius
], layout=Layout(display='none'))

# IDW parameters
idw_power = FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Power:', style={'description_width': 'initial'})
idw_k_neighbors = IntSlider(value=8, min=1, max=50, step=1, description='K Neighbors:', style={'description_width': 'initial'})
idw_radius = FloatSlider(value=10.0, min=0.1, max=100.0, step=0.1, description='Radius (mm):', style={'description_width': 'initial'})
idw_use_radius = Checkbox(value=False, description='Use Radius Limit', style={'description_width': 'initial'})
idw_params = VBox([
    idw_power,
    idw_k_neighbors,
    idw_use_radius,
    idw_radius
], layout=Layout(display='none'))

# KDE parameters
kde_bandwidth = FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Bandwidth:', style={'description_width': 'initial'})
kde_adaptive = Checkbox(value=False, description='Adaptive Bandwidth', style={'description_width': 'initial'})
kde_kernel = Dropdown(
    options=[('Gaussian', 'gaussian'), ('Epanechnikov', 'epanechnikov'), ('Tophat', 'tophat')],
    value='gaussian',
    description='Kernel:',
    style={'description_width': 'initial'}
)
kde_params = VBox([
    kde_bandwidth,
    kde_adaptive,
    kde_kernel
], layout=Layout(display='none'))

# RBF parameters
rbf_kernel = Dropdown(
    options=[
        ('Gaussian', 'gaussian'),
        ('Multiquadric', 'multiquadric'),
        ('Inverse Multiquadric', 'inverse_multiquadric'),
        ('Thin Plate Spline', 'thin_plate_spline'),
        ('Linear', 'linear'),
        ('Cubic', 'cubic'),
        ('Quintic', 'quintic')
    ],
    value='gaussian',
    description='Kernel:',
    style={'description_width': 'initial'}
)
rbf_epsilon = FloatSlider(
    value=1.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Epsilon:',
    style={'description_width': 'initial'}
)
rbf_smoothing = FloatSlider(
    value=0.0,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Smoothing:',
    style={'description_width': 'initial'}
)
rbf_auto_epsilon = Checkbox(
    value=False,
    description='Auto Epsilon',
    style={'description_width': 'initial'}
)
rbf_params = VBox([
    rbf_kernel,
    rbf_auto_epsilon,
    rbf_epsilon,
    rbf_smoothing
], layout=Layout(display='none'))

def update_method_params(change):
    """Show/hide parameter controls based on selected method."""
    method = change['new']
    nn_params.layout.display = 'none'
    linear_params.layout.display = 'none'
    idw_params.layout.display = 'none'
    kde_params.layout.display = 'none'
    rbf_params.layout.display = 'none'
    
    if method == 'nearest':
        nn_params.layout.display = 'flex'
    elif method == 'linear':
        linear_params.layout.display = 'flex'
    elif method == 'idw':
        idw_params.layout.display = 'flex'
    elif method == 'gaussian_kde':
        kde_params.layout.display = 'flex'
    elif method == 'rbf':
        rbf_params.layout.display = 'flex'

method_selector.observe(update_method_params, names='value')
update_method_params({'new': method_selector.value})  # Initialize

# Signal Selector (for visualization after mapping)
signal_label = widgets.HTML("<b>Signal to Visualize:</b>")
signal_selector = Dropdown(
    options=[('Temperature', 'temperature'), ('Power', 'power'), ('Density', 'density')],
    value='temperature',
    description='Signal:',
    style={'description_width': 'initial'}
)

# Data Info
data_info_label = widgets.HTML("<b>Data Info:</b>")
data_info = widgets.HTML("No data generated yet")

left_panel = VBox([
    params_label,
    nn_params,
    linear_params,
    idw_params,
    kde_params,
    rbf_params,
    signal_label,
    signal_selector,
    data_info_label,
    data_info
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Visualization
# ============================================

viz_mode = RadioButtons(
    options=[('2D Slice', '2d'), ('3D View', '3d'), ('Comparison', 'compare')],
    value='2d',
    description='View:',
    style={'description_width': 'initial'}
)

# 2D Slice visualization
slice_axis = RadioButtons(
    options=[('XY', 'xy'), ('XZ', 'xz'), ('YZ', 'yz')],
    value='xy',
    description='Slice Axis:',
    style={'description_width': 'initial'}
)

slice_position = IntSlider(value=50, min=0, max=100, step=1, description='Position (%):', style={'description_width': 'initial'})

viz_output = Output(layout=Layout(height='500px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Visualization</h3>"),
    viz_mode,
    slice_axis,
    slice_position,
    viz_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Results and Metrics
# ============================================

# Mapping Results
results_label = widgets.HTML("<b>Mapping Results:</b>")
results_display = widgets.HTML("No mapping performed yet")
results_section = VBox([
    results_label,
    results_display
], layout=Layout(padding='5px'))

# Performance Metrics
metrics_label = widgets.HTML("<b>Performance Metrics:</b>")
metrics_display = widgets.HTML("No metrics available")
metrics_section = VBox([
    metrics_label,
    metrics_display
], layout=Layout(padding='5px'))

# Method Comparison
comparison_label = widgets.HTML("<b>Method Comparison:</b>")
comparison_display = widgets.HTML("No comparison available")
comparison_section = VBox([
    comparison_label,
    comparison_display
], layout=Layout(padding='5px'))

right_panel = VBox([
    results_section,
    metrics_section,
    comparison_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Progress
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to map signals')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Signal mapping logs output
mapping_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with mapping_logs:
    display(HTML("<p><i>Signal mapping logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Error display (kept for backward compatibility)
error_display = widgets.HTML("")

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Signal Mapping Logs:</b>"),
    mapping_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Keep old status_display for backward compatibility (will be updated by logging functions)
status_display = current_operation

# Global time tracking
operation_start_time = None

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the signal mapping logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with mapping_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Helper Functions
# ============================================

def update_grid_dropdown(change=None):
    """Update grid dropdown when model is selected - show only EMPTY grids filtered by selected data source."""
    global grid_options, current_points, current_signals, current_grid, mapped_grid, current_model_id, current_grid_id, loaded_grid_data, sample_data_generated
    
    # Get model_id from change or current selection
    if change is not None:
        model_id = change['new']
    else:
        model_id = model_dropdown.value
    
    # Set current_model_id so load_mapped_grid can use it
    current_model_id = model_id
    
    # Clear all previous data when model changes to show fresh state
    current_points = None
    current_signals = {}
    current_grid = None
    mapped_grid = None
    current_grid_id = None
    loaded_grid_data = None
    sample_data_generated = False
    
    # Reset grid dropdown
    grid_options = [("━━━ Select Grid ━━━", None)]
    grid_dropdown.value = None
    
    # Reset mapped grid dropdown
    mapped_grid_options = [("━━━ Select Mapped Grid ━━━", None)]
    mapped_grid_dropdown.value = None
        
    # Reset displays to show fresh state
    signal_selector.options = [("No signals", None)]
    signal_selector.value = None
    results_display.value = "<p>No data loaded</p>"
    metrics_display.value = "<p>No data loaded</p>"
    with viz_output:
        clear_output(wait=False)
    status_display.value = "<b>Status:</b> Ready to load data"
    error_display.value = ""
    progress_bar.value = 0

    if model_id and voxel_storage:
        try:
            # Get ALL grids for the model
            available_grids = voxel_storage.list_grids(model_id=model_id, limit=100)
            
            # Get selected data source for filtering
            selected_source = signal_source_selector.value
            
            # Filter to only EMPTY grids (for signal mapping)
            empty_grids = []
            for g in available_grids:
                metadata = g.get('metadata', {})
                config_metadata = metadata.get('configuration_metadata', {})
                available_signals = g.get('available_signals', [])
                grid_name = g.get('grid_name', '')
                
                # Check if grid is empty (no signals)
                has_no_signals = len(available_signals) == 0 if available_signals else True
                
                # Check stage from metadata
                stage = config_metadata.get('stage', '')
                has_empty_stage = stage == 'empty' or 'empty' in grid_name
                
                # Must be an empty grid
                if not (has_no_signals or has_empty_stage):
                    continue
                
                # Filter by data source if not 'all'
                if selected_source != 'all':
                    grid_source = config_metadata.get('source', 'unknown')
                    
                    # Also try to get source from grid name if not in metadata
                    if grid_source == 'unknown':
                        grid_name_lower = grid_name.lower()
                        if 'laser' in grid_name_lower:
                            grid_source = 'laser'
                        elif 'ct' in grid_name_lower:
                            grid_source = 'ct'
                        elif 'ispm' in grid_name_lower:
                            grid_source = 'ispm'
                        elif 'hatching' in grid_name_lower:
                            grid_source = 'hatching'
                    
                    # Only include if source matches
                    if grid_source != selected_source:
                        continue
                
                empty_grids.append(g)
            
            # Build grid options from empty grids
            if not empty_grids:
                if selected_source != 'all':
                    sources_str = selected_source.upper()
                    grid_options.append((f"━━━ No empty grids found for {sources_str} ━━━", None))
                else:
                    grid_options.append(("━━━ No empty grids found ━━━", None))
            else:
                # Group by source for better organization
                grids_by_source = {}
                for g in empty_grids:
                    metadata = g.get('metadata', {})
                    config_metadata = metadata.get('configuration_metadata', {})
                    source = config_metadata.get('source', 'unknown')
                    
                    # Try to get source from grid name if not in metadata
                    if source == 'unknown':
                        grid_name = g.get('grid_name', '')
                        grid_name_lower = grid_name.lower()
                        if 'laser' in grid_name_lower:
                            source = 'laser'
                        elif 'ct' in grid_name_lower:
                            source = 'ct'
                        elif 'ispm' in grid_name_lower:
                            source = 'ispm'
                        elif 'hatching' in grid_name_lower:
                            source = 'hatching'
                    
                    if source not in grids_by_source:
                        grids_by_source[source] = []
                    grids_by_source[source].append(g)
                
                # Add grids grouped by source
                source_order = ['laser', 'ct', 'ispm', 'hatching']
                for source in source_order:
                    if source in grids_by_source:
                        for g in grids_by_source[source]:
                            grid_id = g.get('grid_id', '')
                            grid_name = g.get('grid_name', 'Unknown')
                            metadata = g.get('metadata', {})
                            config_metadata = metadata.get('configuration_metadata', {})
                            grid_type = config_metadata.get('grid_type', 'uniform')
                            resolution = metadata.get('resolution', 0.0)
                            
                            # Format label clearly
                            display_name = f"{source.upper()}: {grid_name}"
                            if resolution > 0:
                                display_name += f" ({grid_type}, {resolution:.1f}mm)"
                            display_name += f" [{grid_id[:8]}...]"
                            
                            grid_options.append((display_name, grid_id))
                
                # Add any grids with unknown source at the end
                if 'unknown' in grids_by_source:
                    for g in grids_by_source['unknown']:
                        grid_id = g.get('grid_id', '')
                        grid_name = g.get('grid_name', 'Unknown')
                        metadata = g.get('metadata', {})
                        config_metadata = metadata.get('configuration_metadata', {})
                        grid_type = config_metadata.get('grid_type', 'uniform')
                        resolution = metadata.get('resolution', 0.0)
                        
                        display_name = f"UNKNOWN: {grid_name}"
                        if resolution > 0:
                            display_name += f" ({grid_type}, {resolution:.1f}mm)"
                        display_name += f" [{grid_id[:8]}...]"
                        
                        grid_options.append((display_name, grid_id))
        
            # Populate mapped grid dropdown with only mapped grids (for loading existing mapped grids)
            mapped_grids = [g for g in available_grids if g.get('available_signals') and len(g.get('available_signals', [])) > 0]
        
            if mapped_grids:
                for g in mapped_grids:
                    grid_id = g.get('grid_id', '')
                    grid_name = g.get('grid_name', 'Unknown')
                    metadata = g.get('metadata', {})
                    available_signals = g.get('available_signals', [])
                
                    # Extract grid type and key info
                    grid_type = metadata.get('grid_type', 'uniform')
                    resolution = metadata.get('resolution', 'N/A')
                    n_signals = len(available_signals) if available_signals else 0
                
                    # Build descriptive label
                    label_parts = [grid_name]
                
                    # Add type info
                    if grid_type != 'uniform':
                        label_parts.append(f"[{grid_type}]")
                
                    # Add resolution
                    if isinstance(resolution, (int, float)):
                        label_parts.append(f"res:{resolution:.1f}mm")
                
                    # Add signal count
                    if n_signals > 0:
                        label_parts.append(f"({n_signals} signal(s))")
                
                    label = " ".join(label_parts)
                    mapped_grid_options.append((label, grid_id))
            
                # Show mapped grid dropdown
                mapped_grid_dropdown.layout.display = 'flex'
            else:
                mapped_grid_options.append(("No mapped grids available", None))
                mapped_grid_dropdown.layout.display = 'flex'
            
        except Exception as e:
            print(f"⚠️ Error loading grids: {e}")
            grid_options.append(("Error loading grids", None))
            mapped_grid_options.append(("Error loading grids", None))
            mapped_grid_dropdown.layout.display = 'flex'
    else:
        # Hide mapped grid dropdown if no model selected
        mapped_grid_dropdown.layout.display = 'none'

    grid_dropdown.options = grid_options
    # Don't auto-select first grid - let user choose
    grid_dropdown.value = None

    mapped_grid_dropdown.options = mapped_grid_options
    mapped_grid_dropdown.value = None

# Add this after the signal_source_selector is defined
def update_grid_dropdown_on_source_change(change):
    """Refresh grid dropdown when signal source changes."""
    if model_dropdown.value:  # Only refresh if model is already selected
        update_grid_dropdown({'new': model_dropdown.value})

# Connect the observer
signal_source_selector.observe(update_grid_dropdown_on_source_change, names='value')

_loading_in_progress = False

def auto_load_data(change):
    """Auto-load data when both model and grid are selected."""
    model_id = model_dropdown.value
    grid_id = grid_dropdown.value
    
    # Auto-load if both model and grid are selected
    if model_id and grid_id:
        load_from_mongodb(None)

model_dropdown.observe(update_grid_dropdown, names='value')
grid_dropdown.observe(auto_load_data, names='value')

def load_from_mongodb(button=None):
    """Load grid and signal data from MongoDB. Can be called manually or auto-triggered."""
    global current_points, current_signals, current_grid, current_model_id, current_grid_id, loaded_grid_data, sample_data_generated, _loading_in_progress, operation_start_time
    
    # Prevent multiple simultaneous loads
    if _loading_in_progress:
        return
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with mapping_logs:
        clear_output(wait=True)
    
    log_message("Starting data load from MongoDB...", 'info')
    update_status("Initializing data load...", 0)
    
    if not unified_client or not voxel_storage or not mongo_client:
        log_message("MongoDB not available. Cannot load data.", 'error')
        error_display.value = "<span style='color: red;'>❌ MongoDB not available. Cannot load data.</span>"
        update_status("MongoDB unavailable", 0)
        return
    
    _loading_in_progress = True
    
    model_id = model_dropdown.value
    grid_id = grid_dropdown.value
    
    if not model_id:
        log_message("Please select a model from the dropdown", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a model from the dropdown</span>"
        update_status("No model selected", 0)
        return
    
    if not grid_id:
        log_message("Please select a grid from the dropdown", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a grid from the dropdown</span>"
        update_status("No grid selected", 0)
        return
    
    log_message(f"Loading data for model {model_id[:8]}... and grid {grid_id[:8]}...", 'info')
    error_display.value = ""
    
    try:
        current_model_id = model_id
        current_grid_id = grid_id
        
        # Load grid
        log_message(f"Loading grid {grid_id[:8]}... from MongoDB...", 'info')
        update_status("Loading grid from MongoDB...", 20)
        loaded_grid_data = voxel_storage.load_voxel_grid(grid_id)
        
        if not loaded_grid_data:
            log_message(f"Failed to load grid {grid_id[:8]}...", 'error')
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid {grid_id[:8]}...</span>"
            update_status("Error loading grid", 0)
            return
        
        log_message("Grid loaded successfully", 'success')
        
        # Reconstruct grid from metadata
        metadata = loaded_grid_data.get('metadata', {})
        bbox_min = tuple(metadata.get('bbox_min', [-50, -50, 0]))
        bbox_max = tuple(metadata.get('bbox_max', [50, 50, 100]))
        resolution = metadata.get('resolution', 2.0)
        
        if VOXEL_AVAILABLE:
            current_grid = VoxelGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                resolution=resolution,
                aggregation=metadata.get('aggregation', 'mean')
            )
        else:
            # Demo grid
            class DemoGrid:
                def __init__(self, bbox_min, bbox_max, resolution):
                    self.bbox_min = np.array(bbox_min)
                    self.bbox_max = np.array(bbox_max)
                    self.resolution = resolution
                    self.size = self.bbox_max - self.bbox_min
                    self.dims = np.ceil(self.size / resolution).astype(int)
                    self.dims = np.maximum(self.dims, [1, 1, 1])
                    self.voxels = {}
                    self.available_signals = set()
            current_grid = DemoGrid(bbox_min, bbox_max, resolution)

        # Use voxel_data if available (for mapped grids), otherwise reconstruct from metadata (for empty grids)
        if loaded_grid_data.get('voxel_data') and VOXEL_AVAILABLE:
            log_message("Using grid structure from loaded voxel data...", 'info')
            update_status("Loading grid structure...", 50)
            # Try to reconstruct from voxel_data
            voxel_data = loaded_grid_data.get('voxel_data')
            # For now, we'll still create a new grid from metadata for consistency
            # In the future, we could deserialize the voxel_data directly
            log_message("Creating grid structure from metadata for signal mapping...", 'info')
        else:
            log_message("Creating grid structure from metadata (empty grid)...", 'info')
        update_status("Preparing grid structure...", 50)
        
        # Check if grid was created from alignment
        # Check both metadata and configuration_metadata for alignment_id
        alignment_id = metadata.get('alignment_id') or metadata.get('configuration_metadata', {}).get('alignment_id')
        created_from_alignment = metadata.get('created_from_alignment', False) or metadata.get('configuration_metadata', {}).get('created_from_alignment', False)
        use_aligned_data = created_from_alignment and alignment_id and alignment_storage
        
        # Load signal data
        if use_aligned_data:
            log_message(f"Loading aligned data from alignment storage (ID: {alignment_id[:8]}...)...", 'info')
            update_status("Loading aligned data from alignment storage...", 60)
        else:
            log_message("Loading signal data from MongoDB...", 'info')
            update_status("Loading signal data from MongoDB...", 60)
        
        signal_source = signal_source_selector.value
        
        # Try to load aligned data first if available
        aligned_data_loaded = False
        if use_aligned_data:
            try:
                alignment = alignment_storage.load_alignment(alignment_id, load_aligned_data=True)
                if alignment and 'aligned_data' in alignment:
                    aligned_data = alignment['aligned_data']
                    aligned_data_loaded = True
                    status_display.value = "<b>Status:</b> ✅ Using aligned data for signal mapping"
            except Exception as e:
                error_display.value = f"<span style='color: orange;'>⚠️ Failed to load aligned data: {e}. Falling back to raw query data.</span>"
                aligned_data_loaded = False
        
        # Query data from MongoDB (fallback or if no aligned data)
        if not aligned_data_loaded:
            all_data = unified_client.get_all_data(model_id)
        
        points_list = []
        signals_dict = {}
        
        # Helper function to extract points and signals from QueryResult
        def extract_data(data, source_name):
            if data is None:
                return
            if hasattr(data, 'points') and data.points is not None:
                points = data.points
                # Convert to numpy array if it's a list
                if isinstance(points, list):
                    points = np.array(points)
                elif not isinstance(points, np.ndarray):
                    points = np.array(points)
                
                # Only add if we have valid points
                if len(points) > 0:
                    points_list.append(points)
                    
                    if hasattr(data, 'signals') and data.signals:
                        for signal_name, signal_values in data.signals.items():
                            # Convert signal values to numpy array if needed
                            if isinstance(signal_values, list):
                                signal_values = np.array(signal_values)
                            elif not isinstance(signal_values, np.ndarray):
                                signal_values = np.array(signal_values)
                            
                            signal_key = f"{source_name}_{signal_name}" if source_name else signal_name
                            if signal_key not in signals_dict:
                                signals_dict[signal_key] = []
                            signals_dict[signal_key].append(signal_values)
        
        # Helper function to extract points and signals from aligned data
        def extract_aligned_data(source_data, source_name):
            if not isinstance(source_data, dict):
                return
            if 'points' in source_data and source_data['points'] is not None:
                points = source_data['points']
                # Convert to numpy array if needed
                if isinstance(points, list):
                    points = np.array(points)
                elif not isinstance(points, np.ndarray):
                    points = np.array(points)
                
                # Only add if we have valid points
                if len(points) > 0:
                    points_list.append(points)
                    
                    # Extract signals if available
                    if 'signals' in source_data and source_data['signals'] is not None:
                        signals = source_data['signals']
                        
                        # Handle both dict (multiple signals) and array (single signal) formats
                        if isinstance(signals, dict):
                            # Multiple signals stored as dict
                            for signal_name, signal_values in signals.items():
                                # Convert signal values to numpy array if needed
                                if isinstance(signal_values, list):
                                    signal_values = np.array(signal_values)
                                elif not isinstance(signal_values, np.ndarray):
                                    signal_values = np.array(signal_values)
                                
                                # Only add if signal has valid length
                                if len(signal_values) > 0:
                                    signal_key = f"{source_name}_{signal_name}" if source_name else signal_name
                                    if signal_key not in signals_dict:
                                        signals_dict[signal_key] = []
                                    signals_dict[signal_key].append(signal_values)
                        elif isinstance(signals, np.ndarray):
                            # Single signal stored as array - use source name as signal name
                            if len(signals) > 0:
                                signal_key = f"{source_name}_signal" if source_name else "signal"
                                if signal_key not in signals_dict:
                                    signals_dict[signal_key] = []
                                signals_dict[signal_key].append(signals)
                        elif isinstance(signals, list):
                            # Convert list to numpy array
                            signals = np.array(signals)
                            if len(signals) > 0:
                                signal_key = f"{source_name}_signal" if source_name else "signal"
                                if signal_key not in signals_dict:
                                    signals_dict[signal_key] = []
                                signals_dict[signal_key].append(signals)
        
        # Load data based on selected source
        if aligned_data_loaded:
            # Use aligned data - this is the preferred method when grid was created from alignment
            sources_extracted = []
            if signal_source in ['laser', 'all'] and 'laser' in aligned_data:
                extract_aligned_data(aligned_data['laser'], 'laser')
                sources_extracted.append('laser')
            
            if signal_source in ['ispm', 'all'] and 'ispm' in aligned_data:
                extract_aligned_data(aligned_data['ispm'], 'ispm')
                sources_extracted.append('ispm')
            
            if signal_source in ['hatching', 'all'] and 'hatching' in aligned_data:
                extract_aligned_data(aligned_data['hatching'], 'hatching')
                sources_extracted.append('hatching')
            
            if signal_source in ['ct', 'all'] and 'ct' in aligned_data:
                extract_aligned_data(aligned_data['ct'], 'ct')
                sources_extracted.append('ct')
            
            # Debug: Show which sources were extracted from aligned data
            if sources_extracted:
                status_display.value = f"<b>Status:</b> ✅ Using aligned data from sources: {', '.join(sources_extracted)}"
        else:
            # Use raw query data (fallback - only if aligned data not available)
            if signal_source in ['laser', 'all']:
                extract_data(all_data.get('laser_parameters'), 'laser')
            
            if signal_source in ['ispm', 'all']:
                extract_data(all_data.get('ispm_monitoring'), 'ispm')
            
            if signal_source in ['hatching', 'all']:
                extract_data(all_data.get('hatching_layers'), 'hatching')
            
            if signal_source in ['ct', 'all']:
                extract_data(all_data.get('ct_scan'), 'ct')
        
        if not points_list:
            if aligned_data_loaded:
                error_display.value = "<span style='color: orange;'>⚠️ No aligned data found for selected source. Using sample data.</span>"
            else:
                error_display.value = "<span style='color: orange;'>⚠️ No signal data found for selected source. Using sample data.</span>"
            # Fallback to sample data
            bbox_min_arr = np.array(bbox_min)
            bbox_max_arr = np.array(bbox_max)
            current_points, current_signals = generate_sample_data(
                n_points=1000,
                bbox_min=tuple(bbox_min_arr),
                bbox_max=tuple(bbox_max_arr)
            )
        else:
            # Convert all points to numpy arrays first
            points_arrays = []
            for p in points_list:
                if isinstance(p, list):
                    p = np.array(p)
                elif not isinstance(p, np.ndarray):
                    p = np.array(p)
                points_arrays.append(p)
            
            # Combine points (handle different shapes)
            if len(points_arrays) > 1:
                # Check if all have same shape
                shapes = [p.shape for p in points_arrays]
                if all(s == shapes[0] for s in shapes):
                    current_points = np.vstack(points_arrays)
                else:
                    # Concatenate if different shapes
                    current_points = np.concatenate(points_arrays, axis=0)
            else:
                current_points = points_arrays[0]
            
            # Ensure points is numpy array and 2D (N, 3)
            if not isinstance(current_points, np.ndarray):
                current_points = np.array(current_points)
            
            if current_points.ndim == 1:
                current_points = current_points.reshape(-1, 3)
            elif current_points.ndim == 2 and current_points.shape[1] != 3:
                # Try to reshape if wrong shape
                if current_points.size % 3 == 0:
                    current_points = current_points.reshape(-1, 3)
                else:
                    error_display.value = f"<span style='color: red;'>❌ Points array has incompatible shape: {current_points.shape}</span>"
                    return
            
            # Combine signals
            current_signals = {}
            for signal_name, signal_arrays in signals_dict.items():
                # Convert all to numpy arrays
                arrays = []
                for arr in signal_arrays:
                    if isinstance(arr, list):
                        arr = np.array(arr)
                    elif not isinstance(arr, np.ndarray):
                        arr = np.array(arr)
                    arrays.append(arr)
                
                if len(arrays) > 1:
                    # Concatenate arrays
                    try:
                        current_signals[signal_name] = np.concatenate(arrays)
                    except:
                        # If shapes don't match, try to flatten and concatenate
                        flattened = []
                        for arr in arrays:
                            if isinstance(arr, np.ndarray):
                                if arr.ndim > 1:
                                    flattened.append(arr.flatten())
                                else:
                                    flattened.append(arr)
                            else:
                                flattened.append(np.array(arr).flatten())
                        current_signals[signal_name] = np.concatenate(flattened)
                else:
                    current_signals[signal_name] = arrays[0] if arrays else np.array([])
            
            # Ensure signal arrays match point count
            n_points = len(current_points)
            for signal_name in list(current_signals.keys()):
                signal_array = current_signals[signal_name]
                if len(signal_array) != n_points:
                    # Truncate or pad to match
                    if len(signal_array) > n_points:
                        current_signals[signal_name] = signal_array[:n_points]
                    else:
                        # Pad with last value
                        padding = np.full(n_points - len(signal_array), signal_array[-1] if len(signal_array) > 0 else 0.0)
                        current_signals[signal_name] = np.concatenate([signal_array, padding])
            
            # Check if signals were extracted
            if not current_signals:
                # If no signals found but we have points
                if aligned_data_loaded:
                    # Aligned data should have signals - if not, user needs to re-run alignment
                    error_display.value = """
                    <span style='color: red;'>
                    <b>❌ Aligned data has points but no signals.</b><br>
                    <b>Action Required:</b> Please go to Notebook 02 (Temporal and Spatial Alignment) and:<br>
                    1. Re-run the alignment<br>
                    2. Click "Save Alignment" to save aligned data WITH signals<br>
                    3. Then return here to map signals<br><br>
                    <b>Note:</b> We cannot mix aligned points with raw query signals - this would break the alignment workflow.
                    </span>
                    """
                    status_display.value = "<b>Status:</b> <span style='color: red;'>Error: Aligned data missing signals</span>"
                    progress_bar.value = 0
                    _loading_in_progress = False
                    return
                else:
                    # For raw query data, this is expected if no signals in query results
                    error_display.value = "<span style='color: orange;'>⚠️ No signal data found for selected source. Using sample data.</span>"
                    # Fallback to sample data
                    bbox_min_arr = np.array(bbox_min)
                    bbox_max_arr = np.array(bbox_max)
                    current_points, current_signals = generate_sample_data(
                        n_points=1000,
                        bbox_min=tuple(bbox_min_arr),
                        bbox_max=tuple(bbox_max_arr)
                    )
                    progress_bar.value = 90
                    sample_data_generated = True
                    
                    # Update data info
                    signal_names = list(current_signals.keys())
                    signal_names_display = ', '.join(signal_names[:5])
                    if len(signal_names) > 5:
                        signal_names_display += f", ... and {len(signal_names) - 5} more"
                    
                    data_info.value = f"""
                    <p><b>Model:</b> {loaded_grid_data.get('model_name', 'Unknown')}</p>
                    <p><b>Grid:</b> {loaded_grid_data.get('grid_name', 'Unknown')}</p>
                    <p><b>Data Source:</b> Sample Data (no signals found in query)</p>
                    <p><b>Points:</b> {len(current_points):,}</p>
                    <p><b>Signals ({len(signal_names)}):</b> {signal_names_display}</p>
                    <p><b>Grid BBox:</b> {bbox_min} to {bbox_max}</p>
                    <p><b>Grid Resolution:</b> {resolution} mm</p>
                    """
                    
                    # Update signal selector
                    available_signals = list(current_signals.keys())
                    if available_signals:
                        signal_selector.options = [(s.capitalize(), s) for s in available_signals]
                        signal_selector.value = available_signals[0]
                    
                    progress_bar.value = 100
                    status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ Using sample data (no signals in query results)</span>"
                    error_display.value = f"<span style='color: orange;'>⚠️ Loaded {len(current_points):,} points with {len(current_signals)} sample signal(s)</span>"
                    _loading_in_progress = False
                    return
        
        # Only proceed if we have signals (already handled the no-signals case above)
        if not current_signals:
            # This should not happen here if the check above worked, but just in case
            _loading_in_progress = False
            return
        
        progress_bar.value = 90
        sample_data_generated = True
        
        # Update data info
        signal_names = list(current_signals.keys())
        signal_names_display = ', '.join(signal_names[:5])
        if len(signal_names) > 5:
            signal_names_display += f", ... and {len(signal_names) - 5} more"
        
        # Show alignment info if using aligned data
        alignment_info = ""
        if aligned_data_loaded and alignment_id:
            alignment_info = f"<p><b>✅ Using Aligned Data</b> (Alignment ID: {alignment_id[:8]}...)</p>"
        elif created_from_alignment and not aligned_data_loaded:
            alignment_info = f"<p><b>⚠️ Grid from alignment</b> but aligned data could not be loaded</p>"
        
        data_info.value = f"""
        <p><b>Model:</b> {loaded_grid_data.get('model_name', 'Unknown')}</p>
        <p><b>Grid:</b> {loaded_grid_data.get('grid_name', 'Unknown')}</p>
        {alignment_info}
        <p><b>Data Source:</b> {signal_source}</p>
        <p><b>Points:</b> {len(current_points):,}</p>
        <p><b>Signals ({len(signal_names)}):</b> {signal_names_display}</p>
        <p><b>Grid BBox:</b> {bbox_min} to {bbox_max}</p>
        <p><b>Grid Resolution:</b> {resolution} mm</p>
        """

        # Update signal selector with available signals
        available_signals = list(current_signals.keys())
        if available_signals:
            signal_selector.options = [(s.capitalize(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
             
        log_message(f"Data loaded: {len(current_points):,} points with {len(current_signals)} signal(s)", 'success')
        update_status("Processing data...", 90)
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Data load completed in {total_time:.2f}s", 'success')
        
        if aligned_data_loaded:
            log_message("Aligned data loaded successfully", 'success')
            error_display.value = f"<span style='color: green;'>✅ Loaded {len(current_points):,} aligned points with {len(current_signals)} signal(s)</span>"
        else:
            log_message("Data loaded successfully", 'success')
            error_display.value = f"<span style='color: green;'>✅ Loaded {len(current_points):,} points with {len(current_signals)} signal(s)</span>"
        
        update_status("Data loaded successfully", 100)
        
    except Exception as e:
        log_message(f"Error loading data: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error loading data: {str(e)}</span>"
        update_status("Error loading data", 0)
        import traceback
        traceback.print_exc()
    finally:
        _loading_in_progress = False

# ============================================
# Mapping Functions
# ============================================

def generate_sample(button):
    """Generate sample point cloud data."""
    global current_points, current_signals, sample_data_generated, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with mapping_logs:
        clear_output(wait=True)
    
    log_message("Generating sample data...", 'info')
    update_status("Generating sample data...", 0)
    
    try:
        # Generate data
        bbox_min = (-50.0, -50.0, 0.0)
        bbox_max = (50.0, 50.0, 100.0)
        log_message(f"Generating 1000 points in bounding box {bbox_min} to {bbox_max}...", 'info')
        current_points, current_signals = generate_sample_data(n_points=1000, bbox_min=bbox_min, bbox_max=bbox_max)
        
        sample_data_generated = True
        
        # Update data info
        data_info.value = f"""
        <p><b>Points:</b> {len(current_points):,}</p>
        <p><b>Signals:</b> {', '.join(current_signals.keys())}</p>
        <p><b>BBox:</b> {bbox_min} to {bbox_max}</p>
        """
        
        log_message(f"Sample data generated: {len(current_points):,} points with {len(current_signals)} signal(s)", 'success')
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Sample data generation completed in {total_time:.2f}s", 'success')
        
        # Update visualization
        update_visualization()
        
        update_status("Sample data generated", 100)
        
    except Exception as e:
        log_message(f"Error generating data: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        update_status("Error generating data", 0)

def map_signals(button):
    """Map ALL signals to voxel grid using selected method."""
    global current_points, current_signals, current_grid, mapped_grid, current_model_id, operation_start_time, sample_data_generated
    
    try:
        # Initialize timing
        operation_start_time = time.time()
        
        # Clear logs
        with mapping_logs:
            clear_output(wait=True)
        
        log_message("Starting signal mapping operation...", 'info')
        log_message(f"Debug: sample_data_generated = {sample_data_generated}", 'info')
        log_message(f"Debug: current_points is None = {current_points is None}", 'info')
        log_message(f"Debug: current_grid is None = {current_grid is None}", 'info')
        log_message(f"Debug: current_signals = {list(current_signals.keys()) if current_signals else 'None'}", 'info')
        update_status("Initializing signal mapping...", 0)

        # Check if data was loaded (always MongoDB mode)
        if not sample_data_generated:
            log_message("⚠️ sample_data_generated is False. Checking if data is actually loaded...", 'warning')
            # Check if we have data even if flag is False
            if current_points is not None and current_grid is not None and current_signals:
                log_message("Data is loaded but flag not set. Proceeding anyway...", 'info')
                sample_data_generated = True  # Set it now
            else:
                log_message("Data not loaded. Grid selection should auto-load data. Please select a grid.", 'warning')
                error_display.value = "<span style='color: red;'>⚠️ Data not loaded. Please select a grid from the dropdown.</span>"
                update_status("Please select a grid", 0)
                return
        
        # Validate data is available (even if sample_data_generated is True)
        if current_points is None:
            log_message("No point data loaded. Please load data first.", 'warning')
            error_display.value = "<span style='color: red;'>⚠️ No point data loaded. Please click 'Load from MongoDB' first.</span>"
            update_status("No point data", 0)
            return
        
        if current_grid is None:
            log_message("No grid loaded. Please load data from MongoDB first.", 'warning')
            error_display.value = "<span style='color: red;'>⚠️ No grid loaded. Please click 'Load from MongoDB' first.</span>"
            update_status("No grid loaded", 0)
            return
        
        if not current_signals:
            log_message("No signals available to map", 'warning')
            error_display.value = "<span style='color: red;'>⚠️ No signals available to map. Please check that data was loaded correctly.</span>"
            update_status("No signals available", 0)
            return
        
        log_message(f"✅ Data validated: {len(current_points):,} points, {len(current_signals)} signal(s)", 'success')
        log_message(f"Signal names: {', '.join(current_signals.keys())}", 'info')
        log_message("Mapping all signals...", 'info')
        error_display.value = ""
        
        # Grid should always be loaded from MongoDB - no fallback needed
        if current_grid is None:
            log_message("❌ No grid loaded. Please load data from MongoDB first.", 'error')
            error_display.value = "<span style='color: red;'>❌ No grid loaded. Please click 'Load from MongoDB' first.</span>"
            update_status("No grid loaded", 0)
            return
        else:
            # Grid already loaded - use its resolution
            log_message(f"Using existing grid with resolution: {current_grid.resolution if hasattr(current_grid, 'resolution') else 'N/A'} mm", 'info')
        
        log_message("Preparing grid for mapping...", 'info')
        if hasattr(current_grid, 'dims'):
            log_message(f"Grid dimensions: {current_grid.dims}", 'info')
        if hasattr(current_grid, 'resolution'):
            log_message(f"Grid resolution: {current_grid.resolution} mm", 'info')
        update_status("Preparing grid...", 20)
        
        # Map ALL signals, not just one
        signals_to_map = current_signals.copy()
        n_signals = len(signals_to_map)
        
        log_message(f"Mapping {n_signals} signal(s): {', '.join(signals_to_map.keys())}", 'info')
        update_status(f"Mapping {n_signals} signal(s)...", 30)
        
        # Perform mapping based on method
        method = method_selector.value
        
        log_message(f"Using interpolation method: {method}", 'info')
        update_status(f"Mapping {n_signals} signal(s) using {method} method...", 40)
        
        if SIGNAL_MAPPING_AVAILABLE and VOXEL_AVAILABLE:
            log_message(f"Signal mapping classes available: SIGNAL_MAPPING_AVAILABLE={SIGNAL_MAPPING_AVAILABLE}, VOXEL_AVAILABLE={VOXEL_AVAILABLE}", 'info')
            # Real mapping - map all signals at once
            start_time = time.time()
            
            try:
                if method == 'nearest':
                    log_message("Running nearest neighbor interpolation...", 'info')
                    update_status("Running nearest neighbor interpolation...", 50)
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='nearest'
                    )
                elif method == 'linear':
                    log_message(f"Running linear interpolation (k={linear_k_neighbors.value})...", 'info')
                    update_status(f"Running linear interpolation (k={linear_k_neighbors.value})...", 50)
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='linear',
                        k_neighbors=linear_k_neighbors.value,
                        radius=linear_radius.value if linear_use_radius.value else None
                    )
                elif method == 'idw':
                    log_message(f"Running IDW interpolation (power={idw_power.value})...", 'info')
                    update_status(f"Running IDW interpolation (power={idw_power.value})...", 50)
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='idw',
                        power=idw_power.value,
                        k_neighbors=idw_k_neighbors.value,
                        radius=idw_radius.value if idw_use_radius.value else None
                    )
                elif method == 'gaussian_kde':
                    log_message(f"Running Gaussian KDE interpolation (bandwidth={kde_bandwidth.value})...", 'info')
                    update_status(f"Running Gaussian KDE interpolation (bandwidth={kde_bandwidth.value})...", 50)
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='gaussian_kde',
                        bandwidth=kde_bandwidth.value,
                        adaptive=kde_adaptive.value
                    )
                elif method == 'rbf':
                    log_message(f"Running RBF interpolation (kernel={rbf_kernel.value})...", 'info')
                    update_status(f"Running RBF interpolation (kernel={rbf_kernel.value})...", 50)
                    mapped_grid = interpolate_to_voxels(
                        current_points, signals_to_map, current_grid,
                        method='rbf',
                        kernel=rbf_kernel.value,
                        epsilon=None if rbf_auto_epsilon.value else rbf_epsilon.value,
                        smoothing=rbf_smoothing.value
                    )
                
                elapsed_time = time.time() - start_time
                log_message(f"✅ Interpolation completed in {elapsed_time:.2f} seconds", 'success')
                update_status("Interpolation completed", 90)
                
            except Exception as e:
                elapsed_time = time.time() - start_time
                log_message(f"❌ Mapping failed after {elapsed_time:.2f}s: {str(e)}", 'error')
                import traceback
                log_message(f"Traceback: {traceback.format_exc()}", 'error')
                error_display.value = f"<span style='color: red;'>❌ Error mapping signals: {str(e)}</span>"
                update_status("Error mapping signals", 0)
                return
        else:
            # Demo mapping - create synthetic mapped data
            log_message(f"⚠️ Using demo mode - SIGNAL_MAPPING_AVAILABLE={SIGNAL_MAPPING_AVAILABLE}, VOXEL_AVAILABLE={VOXEL_AVAILABLE}", 'warning')
            mapped_grid = current_grid
            # Add available_signals to demo grid
            if hasattr(mapped_grid, 'available_signals'):
                mapped_grid.available_signals = set(signals_to_map.keys())
            log_message("Demo mapping completed (no actual interpolation performed)", 'info')
        
        progress_bar.value = 80
        
        # Update signal selector with mapped signals
        if hasattr(mapped_grid, 'available_signals') and mapped_grid.available_signals:
            available_signals = sorted(list(mapped_grid.available_signals))
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
            log_message(f"Updated signal selector with {len(available_signals)} mapped signal(s)", 'info')
        elif signals_to_map:
            # Fallback to original signals
            available_signals = sorted(list(signals_to_map.keys()))
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in available_signals]
            signal_selector.value = available_signals[0]
        
        # Update displays
        update_results_display()
        update_visualization()
        
        # Show save button
        save_mapped_button.layout.display = 'flex'
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"✅ {n_signals} signal(s) mapped successfully in {total_time:.2f}s", 'success')
        else:
            log_message(f"✅ {n_signals} signal(s) mapped successfully", 'success')
        
        error_display.value = f"<span style='color: green;'>✅ Mapped {n_signals} signal(s) to grid</span>"
        update_status(f"{n_signals} signal(s) mapped successfully", 100)
        
    except Exception as e:
        # Catch any exception that happens before logging
        try:
            with mapping_logs:
                print(f"[ERROR] Exception in map_signals: {str(e)}")
                import traceback
                print(traceback.format_exc())
        except:
            # If even logging fails, print to console
            print(f"CRITICAL ERROR in map_signals: {str(e)}")
            import traceback
            traceback.print_exc()
        
        try:
            log_message(f"❌ Error mapping signals: {str(e)}", 'error')
            import traceback
            log_message(f"Traceback: {traceback.format_exc()}", 'error')
            error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
            update_status("Error mapping signals", 0)
        except:
            pass
    finally:
        # Cleanup: Close Spark session if one was created during mapping
        if SPARK_AVAILABLE:
            try:
                from pyspark.sql import SparkSession
                spark = SparkSession.getActiveSession()
                if spark is not None:
                    pass  # Leave session open for reuse
            except Exception:
                pass  # Ignore errors during cleanup
                
def update_results_display():
    """Update results and metrics displays."""
    global mapped_grid, current_grid
    
    if mapped_grid is None:
        return
    
    # Results
    if hasattr(mapped_grid, 'dims'):
        dims = mapped_grid.dims
        total_voxels = int(np.prod(dims))
        
        # Calculate filled voxels from signal arrays (for loaded grids) or voxels dict (for newly mapped)
        filled_voxels = 0
        if hasattr(mapped_grid, '_signal_arrays') and mapped_grid._signal_arrays:
            # Count non-zero voxels from first signal array
            first_signal = list(mapped_grid._signal_arrays.values())[0]
            if isinstance(first_signal, np.ndarray):
                filled_voxels = np.count_nonzero(first_signal)
        elif hasattr(mapped_grid, 'voxels') and mapped_grid.voxels:
            filled_voxels = len(mapped_grid.voxels)
        
        results_html = f"""
        <p><b>Grid Dimensions:</b> {dims[0]} × {dims[1]} × {dims[2]}</p>
        <p><b>Total Voxels:</b> {total_voxels:,}</p>
        <p><b>Filled Voxels:</b> {filled_voxels:,}</p>
        <p><b>Coverage:</b> {100 * filled_voxels / total_voxels if total_voxels > 0 else 0:.1f}%</p>
        """
    else:
        results_html = "<p>Results not available</p>"
    
    results_display.value = results_html
    
    # Metrics
    method = method_selector.value
    # Get available signals count
    n_signals = 0
    if hasattr(mapped_grid, 'available_signals'):
        n_signals = len(mapped_grid.available_signals)
    elif hasattr(mapped_grid, 'voxels'):
        # Try to infer from voxels structure
        n_signals = 1  # Default
    
    # Get resolution from mapped_grid first, then current_grid, then fallback
    resolution = 'N/A'
    if mapped_grid and hasattr(mapped_grid, 'resolution'):
        resolution = mapped_grid.resolution
    elif current_grid and hasattr(current_grid, 'resolution'):
        resolution = current_grid.resolution
    
    metrics_html = f"""
    <p><b>Method:</b> {method}</p>
    <p><b>Resolution:</b> {resolution} mm</p>
    <p><b>Signals Mapped:</b> {n_signals}</p>
    <p><b>Current Signal:</b> {signal_selector.value}</p>
    """
    
    if method == 'linear':
        metrics_html += f"<p><b>K Neighbors:</b> {linear_k_neighbors.value}</p>"
    elif method == 'idw':
        metrics_html += f"<p><b>Power:</b> {idw_power.value}</p>"
        metrics_html += f"<p><b>K Neighbors:</b> {idw_k_neighbors.value}</p>"
    elif method == 'gaussian_kde':
        metrics_html += f"<p><b>Bandwidth:</b> {kde_bandwidth.value}</p>"
    elif method == 'rbf':
        metrics_html += f"<p><b>Kernel:</b> {rbf_kernel.value}</p>"
        metrics_html += f"<p><b>Epsilon:</b> {'Auto' if rbf_auto_epsilon.value else rbf_epsilon.value}</p>"
        metrics_html += f"<p><b>Smoothing:</b> {rbf_smoothing.value}</p>"
    
    metrics_display.value = metrics_html

def update_visualization():
    """Update visualization display."""
    global current_points, current_signals, mapped_grid
    
    with viz_output:
        clear_output(wait=True)
        
        # Import visualization module
        try:
            from am_qadf.visualization import SignalGridVisualizer
        except ImportError:
            from IPython.display import display, HTML
            display(HTML("<p style='color: red;'>Error: Could not import SignalGridVisualizer. Please ensure am_qadf is installed.</p>"))
            return
        
        from IPython.display import display, HTML
        import matplotlib.pyplot as plt
        
        # Check visualization mode
        mode = viz_mode.value
        
        # If we have a mapped grid, visualize from it
        if mapped_grid is not None:
            signal_name = signal_selector.value
            
            # Check if signal exists in mapped grid
            if hasattr(mapped_grid, 'available_signals') and signal_name and signal_name in mapped_grid.available_signals:
                try:
                    # Create visualizer
                    visualizer = SignalGridVisualizer(mapped_grid)
                    
                    # Get signal array for info display
                    signal_array = visualizer._get_signal_array(signal_name)
                    if signal_array is None:
                        display(HTML(f"<p>Signal '{signal_name}' data not available in grid</p>"))
                        return
                    
                    # Get grid dimensions for info
                    metadata = visualizer._get_grid_metadata()
                    dims = metadata['dims']
                    
                    # Check if this is a density/CT signal - use special visualization
                    is_density_signal = (
                        'density' in signal_name.lower() or 
                        'ct' in signal_name.lower() or
                        signal_name.lower() in ['density', 'density_values', 'ct_density']
                    )
                    
                    # Check if this is an ISPM/temperature signal - use special visualization
                    is_ispm_signal = (
                        'temperature' in signal_name.lower() or
                        'ispm' in signal_name.lower() or
                        'thermal' in signal_name.lower() or
                        'melt_pool' in signal_name.lower() or
                        signal_name.lower() in ['temperature', 'melt_pool_temperature', 'ispm_temperature', 'thermal']
                    )
                    
                    # CT/Density visualization (grayscale, multiple slices or 3D volume)
                    if is_density_signal:
                        # Use 3D mode if user selected 3D view, otherwise use 2D slices
                        ct_mode = '3d' if mode == '3d' else 'slices'
                        visualizer.render_ct_density(
                            signal_name=signal_name,
                            mode=ct_mode,
                            show_defects=False,  # Set to True if you have defect locations
                            threshold=None,  # Auto-detect threshold for 3D
                            opacity=0.8,  # For 3D mode
                            auto_show=True
                        )
    
                        # Display info
                        view_type = "3D Volume" if ct_mode == '3d' else "2D Slices"
                        display(HTML(f"<p><b>CT/Density Visualization ({view_type})</b></p>"))
                        display(HTML(f"<p><b>Grid Dimensions:</b> {dims}</p>"))
                        display(HTML(f"<p><b>Density Range:</b> {np.min(signal_array):.3f} to {np.max(signal_array):.3f} g/cm³</p>"))
                        return
                    
                    # ISPM/Temperature visualization (hot colormap, thermal views or 3D volume)
                    elif is_ispm_signal:
                        # Use 3D mode if user selected 3D view, otherwise use 2D slices
                        ispm_mode = '3d' if mode == '3d' else 'slices'
                        visualizer.render_ispm_temperature(
                            signal_name=signal_name,
                            mode=ispm_mode,
                            colormap='hot',
                            show_statistics=True,
                            threshold=None,  # Auto-detect threshold for 3D
                            opacity=0.8,  # For 3D mode
                            auto_show=True
                        )
    
                        # Display info
                        view_type = "3D Volume" if ispm_mode == '3d' else "2D Thermal Views"
                        temp_flat = signal_array.flatten()
                        temp_flat = temp_flat[temp_flat > 0]
                        if len(temp_flat) > 0:
                            display(HTML(f"<p><b>ISPM/Temperature Visualization ({view_type})</b></p>"))
                            display(HTML(f"<p><b>Grid Dimensions:</b> {dims}</p>"))
                            display(HTML(f"<p><b>Temperature Range:</b> {np.min(temp_flat):.1f} to {np.max(temp_flat):.1f} K</p>"))
                            display(HTML(f"<p><b>Mean Temperature:</b> {np.mean(temp_flat):.1f} K (±{np.std(temp_flat):.1f} K)</p>"))
                        return
                    
                    # Regular signal visualization (3D or 2D slice)
                    if mode == '3d':
                        # 3D Visualization using PyVista
                        try:
                            plotter = visualizer.render_3d(
                                signal_name=signal_name,
                                colormap='viridis',
                                threshold=None,  # Auto-detect
                                opacity=0.8,
                                show_edges=True,  # Show grid lines
                                show_scalar_bar=True,
                                title=None,
                                auto_show=True
                            )
                            
                            # Display info
                            display(HTML(f"<p><b>Grid Dimensions:</b> {dims}</p>"))
                            display(HTML(f"<p><b>Signal Range:</b> {np.min(signal_array):.3f} to {np.max(signal_array):.3f}</p>"))
                            return
                            
                        except Exception as e:
                            display(HTML(f"<p style='color: orange;'>⚠️ 3D visualization error: {str(e)}</p>"))
                            # Fall through to 2D slice as fallback
                            mode = '2d'
                    
                    if mode == '2d':
                        # 2D Slice mode
                        # Map slice_axis values ('xy', 'xz', 'yz') to PyVista axis names
                        axis_map = {'xy': 'z', 'xz': 'y', 'yz': 'x'}
                        pv_axis = axis_map.get(slice_axis.value, 'z')
                        
                        # Convert slice position from percentage to actual position
                        slice_pos_percent = slice_position.value
                        metadata = visualizer._get_grid_metadata()
                        
                        if pv_axis == 'x':
                            max_pos = metadata['bbox_max'][0]
                            min_pos = metadata['bbox_min'][0]
                            position = min_pos + (slice_pos_percent / 100.0) * (max_pos - min_pos)
                        elif pv_axis == 'y':
                            max_pos = metadata['bbox_max'][1]
                            min_pos = metadata['bbox_min'][1]
                            position = min_pos + (slice_pos_percent / 100.0) * (max_pos - min_pos)
                        else:  # z
                            max_pos = metadata['bbox_max'][2]
                            min_pos = metadata['bbox_min'][2]
                            position = min_pos + (slice_pos_percent / 100.0) * (max_pos - min_pos)
                        
                        try:
                            plotter = visualizer.render_slice(
                                signal_name=signal_name,
                                axis=pv_axis,
                                position=position,
                                colormap='viridis',
                                show_edges=True,  # Show grid lines
                                show_scalar_bar=True,
                                title=None,
                                auto_show=True
                            )
                            
                            # Display info
                            display(HTML(f"<p><b>Grid Dimensions:</b> {dims}</p>"))
                            display(HTML(f"<p><b>Signal Range:</b> {np.min(signal_array):.3f} to {np.max(signal_array):.3f}</p>"))
                            display(HTML(f"<p><b>Slice:</b> {slice_axis.value.upper()} plane at {slice_pos_percent}%</p>"))
                            return
                            
                        except Exception as e:
                            # Fallback to matplotlib
                            display(HTML(f"<p style='color: orange;'>⚠️ PyVista slice error, using matplotlib fallback: {str(e)}</p>"))
                            
                            # Matplotlib fallback for 2D slice
                            axis_map = {'xy': 2, 'xz': 1, 'yz': 0}
                            axis_idx = axis_map.get(slice_axis.value, 2)
                            slice_pos_percent = slice_position.value
                            
                            if axis_idx == 0:
                                max_pos = max(0, dims[0] - 1)
                                slice_pos = int((slice_pos_percent / 100.0) * max_pos) if max_pos > 0 else 0
                                slice_data = signal_array[slice_pos, :, :]
                            elif axis_idx == 1:
                                max_pos = max(0, dims[1] - 1)
                                slice_pos = int((slice_pos_percent / 100.0) * max_pos) if max_pos > 0 else 0
                                slice_data = signal_array[:, slice_pos, :]
                            else:
                                max_pos = max(0, dims[2] - 1)
                                slice_pos = int((slice_pos_percent / 100.0) * max_pos) if max_pos > 0 else 0
                                slice_data = signal_array[:, :, slice_pos]
                            
                            fig, ax = plt.subplots(figsize=(10, 8))
                            im = ax.imshow(slice_data, cmap='viridis', origin='lower', aspect='auto')
                            
                            if slice_axis.value == 'xy':
                                ax.set_xlabel('X (voxels)')
                                ax.set_ylabel('Y (voxels)')
                                title_plane = 'XY'
                            elif slice_axis.value == 'xz':
                                ax.set_xlabel('X (voxels)')
                                ax.set_ylabel('Z (voxels)')
                                title_plane = 'XZ'
                            else:
                                ax.set_xlabel('Y (voxels)')
                                ax.set_ylabel('Z (voxels)')
                                title_plane = 'YZ'
                            
                            ax.set_title(f'{signal_name} - {title_plane} Slice at Position {slice_pos} ({slice_pos_percent}%)')
                            plt.colorbar(im, ax=ax, label=signal_name)
                            plt.tight_layout()
                            plt.show()
                            
                            display(HTML(f"<p><b>Grid Dimensions:</b> {dims}</p>"))
                            display(HTML(f"<p><b>Signal Range:</b> {np.min(signal_array):.3f} to {np.max(signal_array):.3f}</p>"))
                            return
                
                except Exception as e:
                    display(HTML(f"<p style='color: red;'>❌ Visualization error: {str(e)}</p>"))
                    import traceback
                    display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
                    return
            else:
                display(HTML(f"<p>Signal '{signal_name}' not available in mapped grid</p>"))
                return
        
        # Fallback to point cloud visualization
        if not sample_data_generated or current_points is None:
            display(HTML("<p>Load data and map signals to see visualization</p>"))
            return
        
        # Create 2D slice visualization for point cloud
        signal_name = signal_selector.value
        if signal_name not in current_signals:
            display(HTML(f"<p>Signal '{signal_name}' not available</p>"))
            return
        
        signal_values = current_signals[signal_name]
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Left: Point cloud scatter
        ax1 = axes[0]
        scatter = ax1.scatter(
            current_points[:, 0],
            current_points[:, 1],
            c=signal_values,
            cmap='viridis',
            s=10,
            alpha=0.6
        )
        ax1.set_xlabel('X (mm)')
        ax1.set_ylabel('Y (mm)')
        ax1.set_title(f'Point Cloud: {signal_name}')
        plt.colorbar(scatter, ax=ax1, label=signal_name)
        
        # Right: Grid visualization (if mapped)
        ax2 = axes[1]
        im = None  # Initialize im to avoid UnboundLocalError
        
        if mapped_grid is not None:
            try:
                visualizer = SignalGridVisualizer(mapped_grid)
                signal_array = visualizer._get_signal_array(signal_name)
                
                if signal_array is not None:
                    metadata = visualizer._get_grid_metadata()
                    dims = metadata['dims']
                    z_slice = dims[2] // 2 if len(dims) > 2 else 0
                    if len(signal_array.shape) == 3:
                        slice_data = signal_array[:, :, z_slice]
                    else:
                        slice_data = signal_array
                    im = ax2.imshow(slice_data, cmap='viridis', origin='lower', aspect='auto')
                    ax2.set_title(f'Mapped Grid: {signal_name} (Z={z_slice})')
                else:
                    dims = mapped_grid.dims if hasattr(mapped_grid, 'dims') else [50, 50, 50]
                    data = np.zeros((dims[1], dims[0]))
                    im = ax2.imshow(data, cmap='viridis', origin='lower', aspect='auto')
                    ax2.set_title(f'Mapped Grid: {signal_name}')
            except Exception:
                dims = mapped_grid.dims if hasattr(mapped_grid, 'dims') else [50, 50, 50]
                data = np.zeros((dims[1], dims[0]))
                im = ax2.imshow(data, cmap='viridis', origin='lower', aspect='auto')
                ax2.set_title(f'Mapped Grid: {signal_name}')
        else:
            ax2.text(0.5, 0.5, 'Map signals to see grid visualization', 
                    ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('Grid Visualization')
            # Create a dummy image for colorbar when no mapped grid
            dummy_data = np.zeros((10, 10))
            im = ax2.imshow(dummy_data, cmap='viridis', origin='lower', aspect='auto', visible=False)
        
        ax2.set_xlabel('X (voxels)')
        ax2.set_ylabel('Y (voxels)')
        
        # Only create colorbar if im is defined
        if im is not None:
            plt.colorbar(im, ax=ax2, label=signal_name)
        
        plt.tight_layout()
        plt.show()
        
def save_mapped_grid(button):
    """Save the mapped grid with all signals to MongoDB."""
    global mapped_grid, current_model_id, current_grid_id, voxel_storage, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with mapping_logs:
        clear_output(wait=True)
    
    log_message("Starting mapped grid save operation...", 'info')
    update_status("Initializing save...", 0)
    
    if mapped_grid is None:
        log_message("No mapped grid to save. Please map signals first.", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ No mapped grid to save. Please map signals first.</span>"
        update_status("No mapped grid to save", 0)
        return
    
    if not current_model_id:
        log_message("No model selected. Cannot save grid.", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ No model selected. Cannot save grid.</span>"
        update_status("No model selected", 0)
        return
    
    if not voxel_storage:
        log_message("VoxelGridStorage not available.", 'error')
        error_display.value = "<span style='color: red;'>⚠️ VoxelGridStorage not available.</span>"
        update_status("Storage not available", 0)
        return
    
    log_message("Saving mapped grid...", 'info')
    error_display.value = ""
    
    try:
        # Generate grid name using GridNaming convention
        try:
            from am_qadf.voxel_domain import GridNaming, GridSource, GridType
        except ImportError as e:
            log_message(f"❌ GridNaming module not available: {e}", 'error')
            error_display.value = "<span style='color: red;'>❌ GridNaming module not available. Cannot generate proper grid name.</span>"
            update_status("Error: GridNaming not available", 0)
            return
        
        # Get source, grid_type, and resolution from the original empty grid
        if not loaded_grid_data:
            log_message("❌ Original grid data not available. Cannot generate proper grid name.", 'error')
            error_display.value = "<span style='color: red;'>❌ Original grid data not available. Please reload the grid first.</span>"
            update_status("Error: Grid data not available", 0)
            return
        
        original_metadata = loaded_grid_data.get('metadata', {})
        original_config = original_metadata.get('configuration_metadata', {})
        
        # Get source
        source_str = original_config.get('source', '')
        if not source_str:
            log_message("❌ Source not found in grid metadata. Cannot generate proper grid name.", 'error')
            error_display.value = "<span style='color: red;'>❌ Source not found in grid metadata. Cannot save grid.</span>"
            update_status("Error: Source not found", 0)
            return
        
        # Map to GridSource enum
        source_map = {
            'laser': GridSource.LASER.value,
            'ct': GridSource.CT.value,
            'ispm': GridSource.ISPM.value,
            'hatching': GridSource.HATCHING.value
        }
        source = source_map.get(source_str.lower(), source_str)
        
        # Get grid_type
        grid_type_str = original_config.get('grid_type', '')
        if not grid_type_str:
            log_message("❌ Grid type not found in grid metadata. Cannot generate proper grid name.", 'error')
            error_display.value = "<span style='color: red;'>❌ Grid type not found in grid metadata. Cannot save grid.</span>"
            update_status("Error: Grid type not found", 0)
            return
        
        grid_type_map = {
            'uniform': GridType.UNIFORM.value,
            'adaptive': GridType.ADAPTIVE.value,
            'multires': GridType.MULTIRES.value
        }
        grid_type = grid_type_map.get(grid_type_str.lower(), grid_type_str)
        
        # Get resolution from mapped grid
        if not mapped_grid or not hasattr(mapped_grid, 'resolution'):
            log_message("❌ Resolution not found in mapped grid. Cannot generate proper grid name.", 'error')
            error_display.value = "<span style='color: red;'>❌ Resolution not found in mapped grid. Cannot save grid.</span>"
            update_status("Error: Resolution not found", 0)
            return
        
        resolution = mapped_grid.resolution
        
        # Get mapping method from selector
        method = method_selector.value
        
        # Build method parameters dictionary
        method_params = {}
        if method == 'linear':
            method_params['k'] = linear_k_neighbors.value
            if linear_use_radius.value:
                method_params['r'] = linear_radius.value
        elif method == 'idw':
            method_params['power'] = idw_power.value
            method_params['k'] = idw_k_neighbors.value
            if idw_use_radius.value:
                method_params['r'] = idw_radius.value
        elif method == 'gaussian_kde':
            method_params['bandwidth'] = kde_bandwidth.value
            if kde_adaptive.value:
                method_params['adaptive'] = True
        elif method == 'rbf':
            method_params['kernel'] = rbf_kernel.value
            if not rbf_auto_epsilon.value and rbf_epsilon.value:
                method_params['epsilon'] = rbf_epsilon.value
            if rbf_smoothing.value != 0:
                method_params['smoothing'] = rbf_smoothing.value
        
        # Generate name using GridNaming convention with method and parameters
        grid_name = GridNaming.generate_mapped_grid_name_with_method(
            source=source,
            grid_type=grid_type,
            resolution=resolution,
            method=method,
            method_params=method_params if method_params else None
        )
        log_message(f"Generated grid name using GridNaming: {grid_name}", 'info')
        
        # Get model name if available
        model_name = None
        if stl_client:
            try:
                model_info = stl_client.get_model(current_model_id)
                if model_info:
                    model_name = model_info.get('model_name') or model_info.get('filename', 'Unknown')
            except:
                pass
 
        # Get actual grid resolution (always from mapped grid)
        actual_resolution = mapped_grid.resolution if (mapped_grid and hasattr(mapped_grid, 'resolution')) else 2.0
        if actual_resolution == 2.0:
            log_message("⚠️ Using default resolution 2.0mm (resolution not found in grid)", 'warning')

        # Configuration metadata - COMPREHENSIVE (include source, grid_type, resolution)
        config_metadata = {
            # CRITICAL: Source, grid_type, resolution (required for all operations)
            'source': source,
            'grid_type': grid_type,
            'resolution': actual_resolution,  # REQUIRED: Save resolution directly
    
            # Mapping information
            'mapping_method': method,
            'mapped_signals': sorted(list(mapped_grid.available_signals)) if hasattr(mapped_grid, 'available_signals') else [],
    
            # Original grid information
            'original_grid_id': current_grid_id if current_grid_id else None,
            'original_grid_name': loaded_grid_data.get('grid_name', '') if loaded_grid_data else None,
        }

        
        if method == 'linear':
            config_metadata['linear_k_neighbors'] = linear_k_neighbors.value
            config_metadata['linear_radius'] = linear_radius.value if linear_use_radius.value else None
        elif method == 'idw':
            config_metadata['idw_power'] = idw_power.value
            config_metadata['idw_k_neighbors'] = idw_k_neighbors.value
            config_metadata['idw_radius'] = idw_radius.value if idw_use_radius.value else None
        elif method == 'gaussian_kde':
            config_metadata['kde_bandwidth'] = kde_bandwidth.value
            config_metadata['kde_adaptive'] = kde_adaptive.value
        elif method == 'rbf':
            config_metadata['rbf_kernel'] = rbf_kernel.value
            config_metadata['rbf_epsilon'] = None if rbf_auto_epsilon.value else rbf_epsilon.value
            config_metadata['rbf_smoothing'] = rbf_smoothing.value
            config_metadata['rbf_auto_epsilon'] = rbf_auto_epsilon.value
        
        progress_bar.value = 50
        
        # Save grid
        saved_grid_id = voxel_storage.save_voxel_grid(
            model_id=current_model_id,
            grid_name=grid_name,
            voxel_grid=mapped_grid,
            description=f"Mapped grid with {len(config_metadata['mapped_signals'])} signal(s) using {method} method",
            model_name=model_name,
            configuration_metadata=config_metadata
        )
        
        current_grid_id = saved_grid_id
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Mapped grid saved successfully in {total_time:.2f}s (ID: {saved_grid_id[:8]}...)", 'success')
        else:
            log_message(f"Mapped grid saved successfully (ID: {saved_grid_id[:8]}...)", 'success')
        
        error_display.value = f"<span style='color: green;'>✅ Saved grid: {grid_name} (ID: {saved_grid_id[:8]}...)</span>"
        update_status("Mapped grid saved successfully", 100)
        
    except Exception as e:
        log_message(f"Error saving grid: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error saving grid: {str(e)}</span>"
        update_status("Error saving grid", 0)

def load_mapped_grid(button):
    """Load the selected mapped grid from the dropdown."""
    global mapped_grid, current_model_id, current_grid_id, loaded_grid_data, voxel_storage, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with mapping_logs:
        clear_output(wait=True)
    
    log_message("Loading mapped grid...", 'info')
    error_display.value = ""
    
    # Get model_id from dropdown instead of relying on current_model_id
    model_id = model_dropdown.value
    
    if not voxel_storage or not model_id:
        log_message("Please select a model first", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a model first</span>"
        update_status("Please select a model first", 0)
        return
    
    # Set current_model_id for consistency
    current_model_id = model_id
    
    # Get selected grid from dropdown
    grid_id = mapped_grid_dropdown.value
    
    if not grid_id:
        log_message("Please select a mapped grid from the dropdown", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a mapped grid from the dropdown</span>"
        update_status("Please select a mapped grid", 0)
        return
    
    try:
        log_message(f"Loading mapped grid {grid_id[:8]}...", 'info')
        update_status("Loading grid...", 10)
        
        # Load grid
        loaded_grid_data = voxel_storage.load_voxel_grid(grid_id)
         
        if not loaded_grid_data:
            log_message("Failed to load grid", 'error')
            error_display.value = f"<span style='color: red;'>❌ Failed to load grid</span>"
            update_status("Failed to load grid", 0)
            return
        
        log_message("Grid loaded successfully", 'success')
        update_status("Reconstructing grid...", 60)
        
        # Reconstruct grid from loaded data
        metadata = loaded_grid_data.get('metadata', {})
        bbox_min = tuple(metadata.get('bbox_min', [-50, -50, 0]))
        bbox_max = tuple(metadata.get('bbox_max', [50, 50, 100]))
        resolution = metadata.get('resolution', 2.0)
        
        # Get signal arrays and available signals
        signal_arrays = loaded_grid_data.get('signal_arrays', {})
        available_signals_list = loaded_grid_data.get('available_signals', [])
        
        # Get grid dimensions
        dims = metadata.get('dims', None)
        if dims is None:
            # Calculate dims from bbox and resolution
            size = np.array(bbox_max) - np.array(bbox_min)
            dims = np.ceil(size / resolution).astype(int)
            dims = np.maximum(dims, [1, 1, 1])
        
        log_message(f"Grid dimensions: {dims}, Signals: {len(available_signals_list)}", 'info')
        
        if VOXEL_AVAILABLE:
            mapped_grid = VoxelGrid(
                bbox_min=bbox_min,
                bbox_max=bbox_max,
                resolution=resolution,
                aggregation=metadata.get('aggregation', 'mean')
            )
            
            # Load signal arrays into grid
            if signal_arrays:
                # Store signal arrays in the grid
                if not hasattr(mapped_grid, '_signal_arrays'):
                    mapped_grid._signal_arrays = {}
                
                # Load each signal array
                for signal_name, signal_array in signal_arrays.items():
                    if not isinstance(signal_array, np.ndarray):
                        signal_array = np.array(signal_array)
                    
                    # Reshape signal to grid dimensions if needed
                    if signal_array.size == np.prod(dims):
                        signal_reshaped = signal_array.reshape(dims)
                    else:
                        signal_reshaped = signal_array
                    
                    # Store in grid
                    mapped_grid._signal_arrays[signal_name] = signal_reshaped
                    log_message(f"Loaded signal '{signal_name}' into grid", 'info')
                
                # Set available signals
                mapped_grid.available_signals = set(available_signals_list)
                log_message(f"Loaded {len(signal_arrays)} signal array(s) into grid", 'success')
            else:
                mapped_grid.available_signals = set(available_signals_list)
                log_message("No signal arrays found in loaded data", 'warning')
        else:
            # Demo grid
            class DemoGrid:
                def __init__(self, bbox_min, bbox_max, resolution, dims, signal_arrays, available_signals):
                    self.bbox_min = np.array(bbox_min)
                    self.bbox_max = np.array(bbox_max)
                    self.resolution = resolution
                    self.size = self.bbox_max - self.bbox_min
                    self.dims = dims
                    self.voxels = {}
                    self.available_signals = set(available_signals)
                    self._signal_arrays = {}
                    
                    # Load signal arrays
                    for signal_name, signal_array in signal_arrays.items():
                        if not isinstance(signal_array, np.ndarray):
                            signal_array = np.array(signal_array)
                        
                        # Reshape signal to grid dimensions if needed
                        if signal_array.size == np.prod(dims):
                            signal_reshaped = signal_array.reshape(dims)
                        else:
                            signal_reshaped = signal_array
                        
                        self._signal_arrays[signal_name] = signal_reshaped
            
            mapped_grid = DemoGrid(bbox_min, bbox_max, resolution, dims, signal_arrays, available_signals_list)
            log_message(f"Created demo grid with {len(signal_arrays)} signal(s)", 'success')
        
        current_grid_id = grid_id
        
        log_message("Updating displays...", 'info')
        update_status("Updating displays...", 80)
        
        # Update signal selector with available signals
        available_signals = loaded_grid_data.get('available_signals', [])
        if available_signals:
            signal_selector.options = [(s.replace('_', ' ').title(), s) for s in sorted(available_signals)]
            signal_selector.value = sorted(available_signals)[0]
            log_message(f"Updated signal selector with {len(available_signals)} signal(s)", 'info')
        else:
            signal_selector.options = [("No signals", None)]
            signal_selector.value = None
            log_message("No signals available in loaded grid", 'warning')

        # Set current_grid to mapped_grid for resolution display
        current_grid = mapped_grid
        
        # Update displays
        update_results_display()
        update_visualization()
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Mapped grid loaded successfully in {total_time:.2f}s", 'success')
        else:
            log_message("Mapped grid loaded successfully", 'success')
        
        log_message(f"Loaded grid: {loaded_grid_data.get('grid_name', 'Unknown')} with {len(available_signals)} signal(s)", 'success')
        error_display.value = f"<span style='color: green;'>✅ Loaded grid: {loaded_grid_data.get('grid_name', 'Unknown')} with {len(available_signals)} signal(s)</span>"
        update_status("Mapped grid loaded successfully", 100)
        
    except Exception as e:
        log_message(f"Error loading grid: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error loading grid: {str(e)}</span>"
        update_status("Error loading grid", 0)
        _loading_in_progress = False
        
# Connect events
map_button.on_click(map_signals)
save_mapped_button.on_click(save_mapped_grid)
load_mapped_button.on_click(load_mapped_grid)
signal_selector.observe(lambda x: update_visualization(), names='value')
slice_axis.observe(lambda x: update_visualization(), names='value')
slice_position.observe(lambda x: update_visualization(), names='value')
viz_mode.observe(lambda x: update_visualization(), names='value')

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to map signals to voxel grids using various interpolation methods.

### Key Takeaways

1. **Interpolation Methods**: Nearest Neighbor, Linear, IDW, and Gaussian KDE each have different characteristics
2. **Parameter Tuning**: Method parameters significantly affect mapping quality and performance
3. **Method Selection**: Choose method based on data characteristics and requirements
4. **Visualization**: Compare point cloud data with mapped grid results

### Next Steps

Proceed to:
- **04_Temporal_and_Spatial_Alignment.ipynb** - Learn synchronization and alignment
- **05_Data_Correction_and_Processing.ipynb** - Learn geometric correction and signal processing

### Related Resources

- Signal Mapping Module Documentation: `../docs/AM_QADF/05-modules/signal-mapping.md`
- API Reference: `../docs/AM_QADF/06-api-reference/signal-mapping-api.md`
- Examples: `../examples/`
